In [3]:
# tensorflowがgpuを使っているか確認する
import tensorflow as tf
print(tf.test.is_built_with_cuda())

False


# 問題点
ロボットの現在の方向が分からなければ学習ができないのでは？
もしくは操作角度を相対的なものではなく，絶対的なものとするなど。
同じ行動の繰り返しに対して罰則を与えるべきでは？
軌跡を描画する。

In [5]:
from ray.rllib.algorithms.dreamerv3 import DreamerV3Config
from envs.gym_env import MyEnv, MyEnv2
from ray import train, tune, air
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.impala import ImpalaConfig
from ray.rllib.algorithms.algorithm import Algorithm
from ray.air.integrations.wandb import WandbLoggerCallback
from ray.rllib.utils.replay_buffers.replay_buffer import StorageUnit
import os
import torch
import gymnasium as gym
import pprint

def env_creator(env_config):
    return MyEnv()

def env_creator2(env_config):
    return MyEnv2()

tune.register_env("my_env", env_creator)
tune.register_env("my_env2", env_creator2)

# config = (  # 1. Configure the algorithm,
#     DreamerV3Config()
#     .environment('my_env')
#     .training(
#         model_size="XS", # XS, S, M, L, XL
#     )
# )

# https://docs.ray.io/en/latest/rllib/rllib-training.html#specifying-resources

# 64x64x3の画像を入力とする場合の設定
# model_dict = {"conv_filters": [
#             [16, [8, 8], 4],
#             [32, [4, 4], 4],
#             [64, [4, 4], 2],
#             ],
#             #   "use_attention": True,
#             #   "use_lstm": False,
#               }
# 128x128x3の画像を入力とする場合の設定
model_dict = {"conv_filters": [
            [16, [8, 8], 4],
            [32, [4, 4], 3],
            [64, [3, 3], 2],
            [256, [6, 6], 1],
            ],}

replay_buffer_config = {
        "type": "MultiAgentPrioritizedReplayBuffer",
        # Although not necessary, we can modify the default constructor args of
        # the replay buffer here
        "prioritized_replay_alpha": 0.5,
        "storage_unit": StorageUnit.SEQUENCES,
        "replay_burn_in": 20,
        "zero_init_states": True,
    }
exploration_config = {"type": "PerWorkerEpsilonGreedy",}

# config = ImpalaConfig().environment(env="my_env2", disable_env_checking=True, render_env=True).rollouts(num_rollout_workers=9).training(model=model_dict).framework(framework="torch").resources(num_gpus=1)
config = PPOConfig().environment(env="my_env2", disable_env_checking=True, render_env=True).rollouts(num_rollout_workers=9).training(train_batch_size=256, model=model_dict).framework(framework="torch").resources(num_gpus=1)
# config = SACConfig().environment(env=MyEnv2, disable_env_checking=True, render_env=True).rollouts(num_rollout_workers=9).training(model=model_dict).framework(framework="torch").resources(num_gpus=1).exploration(exploration_config=exploration_config)
# config = SACConfig().environment("CartPole-v1", disable_env_checking=True, render_env=True).rollouts(num_rollout_workers=9).training().framework(framework="torch").resources(num_gpus=1).exploration(exploration_config=exploration_config)
# config = DreamerV3Config().environment(env="my_env2", disable_env_checking=True, render_env=True).training(model=model_dict).resources(num_gpus=1)
# Dreamerは複数Agentに対応していない

tuner = tune.Tuner(
    "PPO",  # 2. Specify the algorithm name
    run_config=train.RunConfig(
        stop={#"episode_reward_mean": 20
              "timesteps_total": 10,
              },  # 学習の終了条件
        callbacks=[WandbLoggerCallback(project="Wandb_nearme")]  # コメントアウトを外すとwandbにログが送信される(https://docs.wandb.ai/quickstart)
    ),
    param_space=config
)

results = tuner.fit()


2024-02-26 01:17:22,901	INFO wandb.py:307 -- Already logged into W&B.
2024-02-26 01:20:35,407	WARNING util.py:202 -- The `callbacks.on_trial_result` operation took 5.233 s, which may be a performance bottleneck.
2024-02-26 01:20:35,409	WARNING util.py:202 -- The `process_trial_result` operation took 5.236 s, which may be a performance bottleneck.
2024-02-26 01:20:35,410	WARNING util.py:202 -- Processing trial results took 5.237 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2024-02-26 01:20:35,410	WARNING util.py:202 -- The `process_trial_result` operation took 5.237 s, which may be a performance bottleneck.
2024-02-26 01:20:39,852	INFO tune.py:1042 -- Total run time: 196.96 seconds (192.54 seconds for the tuning loop).


In [ ]:
best_result = results.get_best_result()
print("\nBest performing trial's final reported metrics:\n")

metrics_to_print = [
    "episode_reward_mean",
    "episode_reward_max",
    "episode_reward_min",
    "episode_len_mean",
]
pprint.pprint({k: v for k, v in best_result.metrics.items() if k in metrics_to_print})

print("come here")
loaded_algo = Algorithm.from_checkpoint(best_result.checkpoint)
print("come here")
policy = loaded_algo.get_policy()

env = gym.make("CartPole-v1")
# env = MyEnv()

episode_reward = 0
terminated = truncated = False
obs, info = env.reset()
print("come here")
while not terminated and not truncated:
    env.render()
    action, _, _ = policy.compute_single_action(obs)
    print(f"action :{action}")
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"reward :{reward}")
    print(f"terminated :{terminated}")
    print(f"truncated :{truncated}")
    episode_reward += reward
print(f"Episode reward :{episode_reward}")

c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\ray\rllib\algorithms\algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be 


Best performing trial's final reported metrics:

{'episode_len_mean': 9.571428571428571,
 'episode_reward_max': 16.619508763643715,
 'episode_reward_mean': 1.3438241469634444,
 'episode_reward_min': -4.810336984042478}
come here


(RolloutWorker pid=29584) 2024-02-26 00:27:00,336	WARNING env_runner_v2.py:297 -- Could not import gymnasium.envs.classic_control.rendering! Try `pip install gymnasium[all]`.


2024-02-26 00:27:01,382	INFO trainable.py:164 -- Trainable.setup took 10.266 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2024-02-26 00:27:01,385	WARNING util.py:62 -- Install gputil for GPU system monitoring.
c:\Users\alaleh\anaconda3\envs\rl_env\lib\site-packages\gymnasium\envs\classic_control\cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


come here
come here


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 4